In [1]:
import sys; sys.path.append('../..')
from abslithist import *

In [2]:
C = lltk.load('CanonFiction')
# C.meta

In [11]:
import sys; sys.path.insert(0,'/Users/ryan/github/cadence')
import cadence as cd

In [6]:
txt="""Happy families are all alike; every unhappy family is unhappy in its
own way.
Everything was in confusion in the Oblonskys’ house. The wife had
discovered that the husband was carrying on an intrigue with a French
girl, who had been a governess in their family, and she had announced
to her husband that she could not go on living in the same house with"""

In [9]:
# !pip install statement-extractor


In [13]:
# from statement_extractor import extract_statements
# text = 'He said "I support you."'
# statements = extract_statements(text)
# print(statements)
# # prints [{'speaker': "He", 'statement': "I support you."}

In [12]:
# pip install -U textacy

In [16]:
# -*- coding: utf-8 -*-
import nltk
from nltk.tokenize import WordPunctTokenizer
import re

PUNCTUATION_TO_REMOVE = ['.', ',', '!', '?']

# ATTRIBUTION_WORDS_STEMMED = [u'disse', u'afirmou', u'comentou', u'completou', u'concluiu', u'reforçou', u'admitiu',u'negou', u'apontou']
ATTRIBUTION_WORDS_STEMMED = ['say', 'affirm', 'comment', 'complete', 'conclude', 'reinforce', 'admit','deny', 'point']

PRONOUNS = ['he', 'she']

class Quotation(object):

    def __init__(self, phrase, verb):
        self.phrase = phrase
        self.verb = verb

    def contains(self, sentence):
        word_present = 0
        for word in sentence:
            if word in self.phrase:
                word_present += 1
        return (word_present > len(sentence) - 4)

    def __unicode__(self):
        return self.phrase


def find_quotes(text):
    pat = re.compile(r"\"(.*?)\",\s+(\w+)", re.UNICODE)
    normalized = text.replace(u'\u201c', '"')
    normalized = normalized.replace(u'\u201d', '"')
    matches = pat.findall(normalized)
    return [Quotation(phrase, verb) for phrase, verb in matches]


def annotate_quotes_with_line_numbers(quotes, sentences):
    numbered_sentences = list(enumerate(sentences))
    for quotation in quotes:
        positions = []
        for pos, sentence in numbered_sentences:
            if quotation.contains(sentence):
                positions.append(pos)
        quotation.positions = positions


def fetch_article(url, language='pt'):
    article = newspaper.Article(url, language=language)
    article.download()
    article.parse()
    return article


def ner_candidates(tree, symbol):
    candidate = []
    for production in tree.productions():
        if production.is_nonlexical():
            continue
        if production.lhs().symbol() == symbol:
            candidate.append(" ".join(i[0] for i in production.rhs()))
    return candidate


def heuristic_elect_candidate(candidates):
    names = {}
    for pos, candidate_list in candidates:
        for candidate in candidate_list:
            try:
                names[candidate] = names[candidate] + 1
            except KeyError:
                names[candidate] = 1

    for other_name, count in names.copy().items():
        for name in names:
            if name == other_name:
                continue
            if other_name in name:
                names[name] += count
    won = sorted(((v, k) for k, v in names.items()))[-1][1]
    return won


sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = WordPunctTokenizer()


def text_to_sentences(raw_text):
    sentences = sentence_tokenizer.tokenize(raw_text)
    tokenized_sentences = [word_tokenizer.tokenize(sentence) for sentence in sentences]
    return tokenized_sentences


In [20]:
s="""Bob walked down the street. He said, "What is going on?" """


find_quotes(s)

[]

In [16]:
cd.get_nlp_doc_wordfeat_df(nlp(txt))

,sent_i,word_i,word_str,pos_upos,pos_xpos,pos_degree,pos_number,pos_mood,pos_tense,pos_verbform,pos_person,pos_gender,pos_poss,pos_prontype,pos_definite,pos_case
0,1,1,Happy,ADJ,JJ,Pos,,,,,,,,,,
1,1,2,families,NOUN,NNS,,Plur,,,,,,,,,
2,1,3,are,AUX,VBP,,,Ind,Pres,Fin,,,,,,
3,1,4,all,ADV,RB,,,,,,,,,,,
4,1,5,alike,ADJ,JJ,Pos,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,3,41,in,ADP,IN,,,,,,,,,,,
67,3,42,the,DET,DT,,,,,,,,,Art,Def,
68,3,43,same,ADJ,JJ,Pos,,,,,,,,,,
69,3,44,house,NOUN,NN,,Sing,,,,,,,,,


In [27]:
def sample_text(path_txt, lim_sents=100):
    nlp = cd.get_nlp(pretokenized=False, postag=True, tokenize=True, constituency=False, depparse=False)

    with open(path_txt) as f: txt=f.read()
    txt=txt.replace('\n\n','\n')
    sents = nltk.sent_tokenize(txt)[:lim_sents]
    o=[]
    for sent_i,sent in enumerate(tqdm(sents)):
        sent_doc = nlp(sent)
        sent_df = cd.get_nlp_doc_wordfeat_df(sent_doc)
        o.append(sent_df.assign(sent_i0=sent_i+1))
    return pd.concat(o).fillna('') if len(o) else pd.DataFrame()


In [29]:
df=sample_text(C.t.path_txt)

100%|██████████| 100/100 [00:26<00:00,  3.80it/s]


In [31]:
df.sent_i.nunique()

4

In [20]:
t.paras_i??

Signature:
t.paras_i(
    para_i=[],
    shuffle_paras=False,
    reset_paras=False,
    lim_paras=None,
    progress=True,
    desc='Iterating over paragraphs',
    **kwargs,
)
Docstring: <no docstring>
Source:   
    def paras_i(self,
            para_i=[],
            shuffle_paras=False,
            reset_paras=False,
            lim_paras=None,
            progress=True,
            desc='Iterating over paragraphs',
            **kwargs):
        
        if para_i:
            para_i_l=[para_i] if type(para_i)==int else list(para_i)
        else:
            para_i_l=self._paras_i
        if reset_paras: para_i_l=sorted(list(self.__paras_i))
        if shuffle_paras: random.shuffle(para_i_l)
        if lim_paras: para_i_l=para_i_l[:lim_paras]
        if progress and len(para_i_l)>1: para_i_l=tqdm(para_i_l,desc=desc)
        return para_i_l
File:      ~/github/cadence/cadence/parsers/docs.py
Type:      method
